# OpenStreetMap (OSM) data for Podgorica, Montenegro

In [ ]:
# Loading of packages required for this Notebook

%pip install quackosm folium matplotlib mapclassify ipywidgets --quiet

### Finding OSM data for area of Podgorica (Montenegro)

In [ ]:
import quackosm as qosm

# Finding geometry of Podgorica
area_podgorica = qosm.geocode_to_geometry("Capital Podgorica, Montenegro")
area_podgorica

In [ ]:
# Find suitable PBF file(s) of OSM that covers Podgorica geometry 
# And filter out OSM features only for that area
gdf = qosm.convert_geometry_to_geodataframe(area_podgorica)
gdf

### Explore extracted OSM features

In [ ]:

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import geopandas as gpd

fig = plt.figure(figsize=(10, 10))
ax = fig.subplots()

gdf.plot(ax=ax, markersize=1, zorder=1, alpha=0.4)
gdf.boundary.plot(ax=ax, markersize=0, zorder=1, alpha=0.8)
gpd.GeoSeries([area_podgorica], crs=4326).plot(
    ax=ax,
    color=(0, 0, 0, 0),
    zorder=2,
    hatch="///",
    edgecolor="orange",
    linewidth=1.5,
    
)

blue_patch = mpatches.Patch(color="C0", alpha=0.8, label="Extracted OSM features")
orange_patch = mpatches.Patch(
    facecolor=(0, 0, 0, 0), edgecolor="orange", hatch="///", linewidth=1.5, label="Podgorica Geometry filter"
)
ax.legend(handles=[blue_patch, orange_patch], loc="lower right")
plt.show()

### Explore facilities of education in Podgorica

In [ ]:
import geopandas as gpd

# amenity=bench
filter_1 = {"amenity": "bench"}
filter_vrtic ={"building": "kindergarten", "amenity": "kindergarten"}
filter_school ={"building": "school", "amenity": "school"}
filter_university ={"building": "university", "amenity": "university"}


gdf = qosm.convert_geometry_to_geodataframe(area_podgorica, tags_filter=filter_vrtic, verbosity_mode="transient", keep_all_tags=True, explode_tags=True)
gdf1 = qosm.convert_geometry_to_geodataframe(area_podgorica, tags_filter=filter_school, verbosity_mode="transient", keep_all_tags=True, explode_tags=True)
gdf2 = qosm.convert_geometry_to_geodataframe(area_podgorica, tags_filter=filter_university, verbosity_mode="transient", keep_all_tags=True, explode_tags=True)


#gdf = qosm.convert_geometry_to_geodataframe(geometry, tags_filter={"highway": "tertiary", "building": "school"}, verbosity_mode="silent")
#gdf = qosm.convert_pbf_to_geodataframe(tags_filter={"admin_level": "6", "amenity": "bench"}, pbf_path="files/geofabrik_europe_montenegro.osm.pbf")


m = gdf.explore(color="orangered", tiles="CartoDB positron")
m2 = gdf1.explore(color="blue", m=m)
m3 = gdf2.explore(color="pink", m=m)




gpd.GeoSeries([geometry], crs=4326).boundary.explore(m=m)